In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import convolve2d
from skimage.io import imread

In [ ]:
rootfolder = '.'

Define the function to compute the kernel given the weights and the degree of the polynomial

In [ ]:
def compute_2D_LPA_kernel(w, N):
    # compute the 2D LPA kernel for a given weights and polynomial degree
    # input:
    #   w: matrix containing the weights for the local LS problem
    #   N: degree of the polynomial approximation
    # return:
    #   g: the computed LPA kernel


    # window size is the lenght of the weight vector
    r, c = w.shape
    M = r*c

    # create the matrix T
    tx = np.linspace(0, 1, c)
    ty = np.linspace(0, 1, r)
    tx, ty = np.meshgrid(tx, ty)
    tx = tx.reshape(-1)
    ty = ty.reshape(-1)
    T = np.zeros((M,(N+1)**2))
    cnt = 0
    for i in range(N+1):
        for j in range(N-i+1):
            if i==0 and j==0:
                T[:, cnt] = np.ones(M)
            else:
                T[:, cnt] = tx**i * ty**j
            cnt = cnt + 1
    T = T[:, :cnt]

    # unroll the matrix of the weights
#    w =

    # generate the inverse of weights
#    winv =

    # set to zero weights that are inf
#    winv

    # define the weight matrix
#    W =
#    Winv =

    ## construct the LPA kernel

    # comput the qr decomposition of WT
#    Q, R =

    # define Qtilde
#    Qtilde =

    # adjust Qtilde with the weights matrix squared
#    W2Qtilde =

    # select the central row of W2Qtilde
#    row =

    # compute the kernel
#    g_bar =

    #reshape the kernel in a matrix
#    g_bar =

    # flipping, since it is used in convolution
#    g =

    return g


Load the image and add the noise

In [ ]:
img = imread(f'{rootfolder}/data/cameraman.png') / 255

sigma_noise = 20/255
noisy_img = img + np.random.normal(size=img.shape) * sigma_noise

#psnr_noisy =


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(img, cmap='gray')
ax[0].set_title('Original image')

ax[1].imshow(noisy_img, cmap='gray')
ax[1].set_title(f'Noisy image, PSNR = {psnr_noisy:.2f}')

LPA-ICI 2D
----------
Set the LPA-ICI parameters

In [ ]:
# maximum degree of polynomial used for fitting
N = 1

# parameter for the confidence intervals in the ICI rule
Gamma = 2

# Set all the scale values
hmax = 21
all_h = np.arange(1, hmax+1)

Generate the LPA kernels for all the scales. Use centered weights.


In [ ]:
all_g = []
for i, h in enumerate(all_h):
        # define the weights for the scale h symmetric

        # size of the weight MATRIX
#        w =

        # compute and store the kernel g
        g = compute_2D_LPA_kernel(w, N)
        all_g.append(g)

Initialize all the variables for the ICI rule

In [ ]:
# initialize the estimate for each scale
yhat = np.zeros((img.shape))

# initialize the vector containing the best scale for each sample
best_scale = np.zeros(shape=yhat.shape)

# initialize the lower and upper bound matrices
lower_bounds = - np.inf * np.ones(shape=yhat.shape)
upper_bounds = np.inf * np.ones(shape=yhat.shape)

Loop over all the scales

In [ ]:
for i, h in enumerate(all_h):
    g = all_g[i]

    # compute the estimate for the scale h
#    yhat_h =

    # compute the lower and upper bound of the confidence interval for the scale h
#    lb =
#    ub =

    # update the lower and upper bounds
#    lower_bounds =
#    upper_bounds =

    # identify for which samples h is the best scale according to the
    # ICI rule and update the best_scale vector accordingly
    # update best_scale

    # update the estimate
    # update yhat

Compute the PSNR

In [ ]:
# psnr =

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(yhat, cmap='gray')
ax[0].set_title(f'LPA-ICI estimate, PSNR = {psnr:.2f}')

ax[1].imshow(best_scale)
ax[1].set_title('Best scale for each pixel')
fig.colorbar(ax[1].pcolormesh(best_scale), ax=ax[1])

Anisotropic LPA-ICI
------------------------
Set the parameters


In [ ]:
# maximum degree of polynomial used for fitting
N = 1

# parameter for the confidence intervals in the ICI rule
Gamma = 2

# Set all the scale values
hmax = 21
all_h = np.arange(1, hmax+1)

# set all the direction values
all_theta = np.arange(4)

Generate the LPA kernels for all the scales and all the directions

In [ ]:
all_g = []

for theta in all_theta:

    all_g_theta = []
    for i, h in enumerate(all_h):
        # define the weights for the scale h and the direction theta
#        if theta
#            w =

        # compute and store the kernel g
        g = compute_2D_LPA_kernel(w, N)

        all_g_theta.append(g)

    all_g.append(all_g_theta)


Initialize all the variables

In [ ]:
# initialize the estimate for each scale
yhat = np.zeros(img.shape)

# initialize the matrix of the aggregation weights
weights = np.zeros(img.shape)


Use the LPA-ICI to compute find the best scale for each direction and compute the finale estimates

In [ ]:
# loop over all the directions
for theta in all_theta:
    # initialize the estimate for the direction theta
    yhat_theta = np.zeros(img.shape)

    # initialize the matrix all the variances for the direction theta
    var_theta = np.zeros(img.shape)

    # initialize the lower and upper bounds matrices
    # lower_bounds =
    # upper_bounds =

    # loop over all scales
    all_g_theta = all_g[theta]
    for i, h in enumerate(all_h):
        g = all_g_theta[i]

        # compute the estimate for the scale h
#        yhat_h =

        # compute the lower and upper bound of the confidence interval for the scale h
#        lb =
#        ub =

        # update the lower and upper bounds
#        lower_bounds =
#        upper_bounds =

        # update the estimate
        # update yhat_theta

        # update the matrix with the variances
        # update var_theta

    # update the estimates and the weights
    # yhat =
    # weights =

# compute the final estimates
yhat = yhat / weights

Compute the PSNR

In [ ]:
# psnr =

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(yhat, cmap='gray')
plt.title(f'LPA-ICI estimate, PSNR = {psnr:.2f}')
